In [1]:
filename = 'chat.txt'

In [2]:
with open(filename, 'r') as f:
  lines = f.readlines()

for l in lines[:10]:
  print(l)

13/10/19, 2:03 pm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.

13/10/19, 2:03 pm - +91 86059 36547 created group "MRT aka Painter partition"

13/10/19, 2:03 pm - mehul mathur: Y'all doin mrt

13/10/19, 2:03 pm - mehul mathur: ?

13/10/19, 2:06 pm - Pooja: Ya

13/10/19, 2:14 pm - Manasvi💩: nice

13/10/19, 2:18 pm - mehul mathur: First one accepted gives treat

13/10/19, 3:01 pm - Pooja: I did sx

13/10/19, 3:01 pm - Pooja: Last one

13/10/19, 3:25 pm - mehul mathur: Eyy no



In [4]:
## magic commands
# %reload_ext autoreload
# %autoreload 2
# %matplotlib inline
## import required packages
import pandas as pd
import re

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[",
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text)


## function to parse the whatsapp extract file
def parse_file(text_file):
    """Convert WhatsApp chat log text file to a Pandas dataframe."""

    # Regex to account for messages taking up multiple lines
    pat = re.compile(r'^(\d\d\/\d\d\/\d\d.*?)(?=^^\d\d\/\d\d\/\d\d|\Z)', re.S | re.M)
    with open(text_file, encoding='utf-8', errors='replace') as f:
        data = [m.group(1).strip().replace('\n', ' ') for m in pat.finditer(f.read())]

    sender = []
    message = []
    datetime_raw = []
    print(len(data))

    for row in data:
        # timestamp is before the first dash
        datetime_raw.append(row.split(' - ')[0])

        # sender is between dash and colon
        try:
            s = re.search(r' - (.*?):', row).group(1)
            sender.append(s.strip())
        except Exception:
            sender.append('')

        # message content is after the first colon
        try:
            msg = row.split(': ', 1)[1]
            msg = re.sub(r'@\d{10,}', '', msg)  # tagged numbers
            msg = re.sub(r'<Media omitted>', '', msg)
            msg = re.sub(r'You deleted this message', '', msg)
            msg = re.sub(r'This message was deleted', '', msg)
            msg = re.sub(r'https?://\S+|www\.\S+', '', msg)
            msg = re.sub(r'\s+', ' ', msg).strip()  # keep emojis/punctuation; normalize whitespace
            message.append(msg)
        except Exception:
            message.append('')

    df = pd.DataFrame(zip(datetime_raw, sender, message), columns=['timestamp', 'sender', 'text'])

    # Robust timestamp parsing: tolerate format variations; drop rows that fail parsing
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', dayfirst=True, infer_datetime_format=True)
    df = df.dropna(subset=['timestamp']).reset_index(drop=True)

    # remove events not associated with a sender
    df = df[df.sender != ''].reset_index(drop=True)
    df = df[df['text'].str.split().str.len() > 1].reset_index(drop=True)

    return df

## Data prep upgrade 5: don’t drop messages + keep emojis
Two critical fixes for learning a realistic WhatsApp style:

1) **Timestamp parsing is now tolerant**
- Removed the `len()==17` filter (it was silently dropping valid rows).
- Parse timestamps with `pd.to_datetime(..., errors='coerce')` and drop only the rows that truly fail parsing.

2) **Messages keep emojis and most punctuation**
- Emojis carry strong style/tone signal (laughing, sarcasm, excitement).
- We still remove obvious noise (URLs, media omitted, deleted-message placeholders), but we don’t strip emojis out of the actual message text anymore.

In [7]:
df = parse_file(filename)

33732


In [8]:
df

,timestamp,sender,text
0,2019-10-13 14:03:00,mehul mathur,Yall doin mrt
1,2019-10-13 14:18:00,mehul mathur,First one accepted gives treat
2,2019-10-13 15:01:00,Pooja,I did sx
3,2019-10-13 15:01:00,Pooja,Last one
4,2019-10-13 15:25:00,mehul mathur,Eyy no
...,...,...,...
17246,2022-12-27 09:31:00,Pooja,Now we re taking no
17247,2022-12-27 09:33:00,mehul mathur,We didnt get beh
17248,2022-12-27 09:33:00,mehul mathur,We have to take molecular one
17249,2022-12-31 13:02:00,mehul mathur,This is child endangerment?


In [9]:
## list of conversation participants
participants = list(df['sender'].str.lower().
                    str.replace(' ', '_').unique())
participants

['mehul_mathur', 'pooja', 'manasvi']

In [ ]:
## concatenate names and text into one string (now with explicit message boundaries)
sender_norm = df['sender'].str.replace(' ', '_')
msg = df['text']
text = [(sender_norm + ' ' + msg + ' xxeom').str.cat(sep=' ')]

In [11]:
# inspect
text[0][5000:6000]

' Manasvi Thats the group assessment Manasvi Quiz is individual only mehul_mathur no no i meant mehul_mathur we help mehul_mathur each other Manasvi Yeah lol mehul_mathur Bro manasvi mehul_mathur Pushing for expert Manasvi dheere dheere mehul_mathur This dont work mehul_mathur What is the problem 00000000.. then? Manasvi cant be solved? mehul_mathur Also here you are first only assuming that each problem can be solved mehul_mathur arey you have to prove that mehul_mathur you cant say that 0000 means cant be solved mehul_mathur you have to prove that there always some problems that cnt be solved Manasvi it just means with the program i have with me i cant solve it mehul_mathur program you have with you kya? mehul_mathur what programs you have bey mehul_mathur abey chup Manasvi youre making me mad Manasvi idk maybe im seeing it worng mehul_mathur just a little wrong mehul_mathur no i lied Pooja youre right Pooja each problem is subset of power set Pooja ya mehul Manasvi thats what i said

In [12]:
def add_spk(text):
    tokens = text.split()
    res = []
    for t in tokens:
        if t.lower() in participants:
          res.append('xxspk')
          res.append(t.lower())
        else:
          res.append(t)
    return ' '.join(res)


add_spk(text[0][6000:7000])

text[0] = add_spk(text[0])
full_text = text[0]

## Data prep upgrade 2: add `xxeom` (end-of-message) token
Right now the LM sees the entire chat as one long stream. By inserting an explicit end-of-message marker `xxeom` after every message, we give the model a clean, learnable boundary between turns.

What this changes in the training text (conceptually):
- Before: `... xxspk alice <message tokens> xxspk bob <message tokens> ...` (boundaries are implicit)
- After:  `... xxspk alice <message tokens> xxeom xxspk bob <message tokens> xxeom ...` (boundaries are explicit)

Why it helps (especially for chat-completion):
- The model learns *when to stop a message* and transition to the next speaker/turn.
- Generation becomes easier to control: you can generate until `xxeom` to produce exactly one reply instead of rambling.
- Loss curves typically become a bit more stable because the dataset has clearer structure (predictable boundary token).

In [ ]:
def _time_bucket(ts):
    h = ts.hour
    if 5 <= h < 12:
        return 'xxtime_morning'
    if 12 <= h < 17:
        return 'xxtime_afternoon'
    if 17 <= h < 22:
        return 'xxtime_evening'
    return 'xxtime_night'

def _dow_token(ts):
    # mon/tue/... based on pandas day name
    return 'xxdow_' + ts.day_name()[:3].lower()

def _msg_type_token(text):
    t = (text or '').strip()
    if len(t.split()) <= 2:
        return 'xxmsg_short'
    if '?' in t:
        return 'xxmsg_question'
    if '!' in t:
        return 'xxmsg_exclaim'
    return 'xxmsg_statement'

def train_valid_text_by_time(df, frac=0.8):
    """Deterministic split: train on earlier messages, validate on later messages."""
    df_sorted = df.sort_values('timestamp').reset_index(drop=True)
    cut = int(len(df_sorted) * frac)

    def _to_lm_text(frame):
        time_tok = frame['timestamp'].apply(_time_bucket)
        dow_tok = frame['timestamp'].apply(_dow_token)
        msg_tok = frame['text'].apply(_msg_type_token)
        sender_norm = frame['sender'].str.replace(' ', '_')
        merged = (time_tok + ' ' + dow_tok + ' ' + msg_tok + ' ' + sender_norm + ' ' + frame['text'] + ' xxeom').str.cat(sep=' ')
        return add_spk(merged)

    train_text = _to_lm_text(df_sorted.iloc[:cut])
    valid_text = _to_lm_text(df_sorted.iloc[cut:])
    return train_text, valid_text

In [ ]:
train_text, valid_text = train_valid_text_by_time(df, frac=0.8)

## Data prep upgrade 3: time-aware train/valid split
Instead of taking a random substring of the full text, we now split **by timestamp**: earlier messages go to training, later messages go to validation.

Why this is best practice for chat logs:
- It prevents leakage of “future style/topics” into training.
- It makes validation closer to a real deployment scenario: predicting what comes next in time.
- Your `valid_loss`/perplexity becomes much more meaningful (and less dependent on an arbitrary substring boundary).

## Data prep upgrade 4: metadata tokens per message
We now prepend *three lightweight semantic tokens* to every message before the speaker name:
- `xxtime_*`: coarse time-of-day bucket from the timestamp (morning/afternoon/evening/night)
- `xxdow_*`: day-of-week bucket (mon/tue/...)
- `xxmsg_*`: message-type heuristic (question/exclaim/statement/short)

So each message looks like:
`xxtime_morning xxdow_mon xxmsg_question xxspk alice ... xxeom`

Why this helps without changing the core LM:
- These tokens act like **features the model can condition on**, improving consistency and controllability.
- They compress useful context (time + intent) into a few tokens, which is especially valuable with short context windows (e.g., 128 tokens).

In [15]:
print(len(train_text), len(valid_text))

570628 142658


In [16]:
train_df = pd.DataFrame({'text': [train_text]})
valid_df = pd.DataFrame({'text': [valid_text]})

# Combine into one DataFrame with 'is_valid' column
lm_df = pd.concat([train_df, valid_df]).reset_index(drop=True)
lm_df['is_valid'] = [False, True]

In [17]:
lm_df

,text,is_valid
0,god of optimums xxspk mehul_mathur bruteforce gos xxspk mehul_mathur we can have a hell also xxspk mehul_mathur where all worst case shit goes xxspk mehul_mathur like that xxspk pooja but what is a god xxspk pooja Ok greedy god exists xxspk pooja Whats its purpose noe xxspk mehul_mathur he handles all greedy stuff xxspk mehul_mathur arey idea came today morning xxspk mehul_mathur so chill xxspk manasvi that kinda doesnt make sense lmao xxspk pooja Yeah exactly xxspk pooja Like its creative and all xxspk mehul_mathur arey bro xxspk mehul_mathur it just happened xxspk mehul_mathur im saying...,False
1,xxspk mehul_mathur Yall doin mrt xxspk mehul_mathur First one accepted gives treat xxspk pooja I did sx xxspk pooja Last one xxspk mehul_mathur Eyy no xxspk mehul_mathur Who you copy xxspk mehul_mathur And mrt is aggressive cows xxspk mehul_mathur Not painter partition xxspk pooja Yeah no omg xxspk pooja Sasanaka was saying that also xxspk mehul_mathur Did anyone get it xxspk pooja out of the 3 of us xxspk pooja Lez read about cows xxspk pooja Did you read xxspk mehul_mathur Hrishi is trying to explain xxspk mehul_mathur I said I didnt understand xxspk mehul_mathur He said sorry that I exp...,True


## Model upgrade: simple Transformer LM (GPT‑2 style)
From this point on, we’ll use a small GPT‑2–style Transformer (`distilgpt2`) instead of AWD‑LSTM. This gives you subword/BPE tokenization and stronger generation quality while staying lightweight.

In [ ]:
# (Optional) Install if missing. In VS Code/Jupyter, %pip installs into the current kernel.
# %pip -q install transformers accelerate sentencepiece
import math
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    set_seed,
 )

In [ ]:
set_seed(42)
MODEL_NAME = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Make chat + metadata control tokens single, learnable tokens
metadata_tokens = [
    'xxspk', 'xxeom',
    'xxtime_morning', 'xxtime_afternoon', 'xxtime_evening', 'xxtime_night',
    'xxmsg_question', 'xxmsg_exclaim', 'xxmsg_statement', 'xxmsg_short',
    'xxdow_mon', 'xxdow_tue', 'xxdow_wed', 'xxdow_thu', 'xxdow_fri', 'xxdow_sat', 'xxdow_sun',
 ]
tokenizer.add_special_tokens({'additional_special_tokens': metadata_tokens})

# GPT-2 has no pad token by default; map padding to EOS for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
def _token_ids(text, tokenizer):
    return tokenizer(text, add_special_tokens=False)['input_ids']

class GPTBlockDataset(Dataset):
    """Fixed-length blocks for causal LM fine-tuning (labels = input_ids)."""
    def __init__(self, token_ids, block_size=128):
        n_blocks = len(token_ids) // block_size
        trimmed = token_ids[: n_blocks * block_size]
        self.blocks = torch.tensor(trimmed, dtype=torch.long).view(n_blocks, block_size)

    def __len__(self):
        return self.blocks.size(0)

    def __getitem__(self, idx):
        x = self.blocks[idx]
        return {
            'input_ids': x,
            'attention_mask': torch.ones_like(x),
            'labels': x,
        }

block_size = 128
train_ids = _token_ids(train_text, tokenizer)
valid_ids = _token_ids(valid_text, tokenizer)
train_ds = GPTBlockDataset(train_ids, block_size=block_size)
valid_ds = GPTBlockDataset(valid_ids, block_size=block_size)
len(train_ds), len(valid_ds)

In [ ]:
training_args = TrainingArguments(
    output_dir='gpt2_whatsapp',
    overwrite_output_dir=True,
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy='steps',
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    logging_steps=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=5e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    report_to='none',
 )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
 )

In [ ]:
# Train + evaluate
trainer.train()
eval_metrics = trainer.evaluate()
eval_loss = eval_metrics.get('eval_loss')
ppl = math.exp(eval_loss) if eval_loss is not None else None
eval_loss, ppl

### Why this Transformer change encodes “more meaning” (without making the project huge)
This upgrade changes *how the text is represented* and *how context is used*:
- **BPE/subword tokenization** (GPT‑2 tokenizer) handles slang, typos, mixed languages, and names better than word-level vocabularies.
- Adding `xxspk`/`xxeom` as **special tokens** makes speaker turns and message boundaries explicit *and* compact (single tokens), which improves controllability.
- A Transformer is better at using the last ~128 tokens of context than an LSTM at the same training budget, so chat-completion usually feels more coherent.

In [ ]:
def render_generated(text):
    text = (text.replace(' xxspk ', '\n')
                .replace(' xxeom ', '\n')
                .replace(" \\\'", "\\\'")
                .replace(" n\\'t", "n\\'t"))
    text = re.sub(r'\s([?.!\"](?:\s|$))', r'\1', text)
    for participant in participants:
        text = text.replace(participant, participant + ':')
    return text

def generate_chat_gpt2(prompt, max_new_tokens=120, temperature=0.8, top_p=0.9):
    model.eval()
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        out = model.generate(
            **inputs,
            do_sample=True,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.decode(out[0], skip_special_tokens=False)
    return render_generated(decoded)

In [ ]:
# Example prompt: include metadata + last turn + starter for next speaker
prompt = (
    f"xxtime_evening xxdow_fri xxmsg_question xxspk {participants[0]} i am hungry anyone coming to eat ? xxeom "
    f"xxtime_evening xxdow_fri xxmsg_statement xxspk {participants[1]}"
 )
print(generate_chat_gpt2(prompt, max_new_tokens=120))